In [1]:
# Playing around with SAMI data!

import astropy.io.fits as fits
import numpy as np
from astropy.table import unique,Table
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt

# Read in sami cube data
sami_cube_list = Table.read('/suphys/fscu5535/python_scripts/raw_data/cube_list.fits', format='fits',memmap=True)

# Display basic table info
print("Table info:","\n")
sami_cube_list.info()

# Read in all gama data
gama_galaxies = Table.read('/suphys/fscu5535/python_scripts/raw_data/G3CGalv10.fits', format='fits',memmap=True)

# Display basic table info
print("Table info:","\n")
gama_galaxies.info()

print("\n\n")

# Plot a 1D histogram
# groupid_hist = plt.hist(group_finding['RA'], bins='auto')

Table info: 

<Table length=3712>
   name    dtype   unit  n_bad
--------- ------- ------ -----
   CUBEID bytes80            0
CUBEIDPUB bytes16            0
 CUBENAME bytes80            0
    CATID   int64            0
    RFLAG  bytes1            0
 CUBEFWHM float32 arcsec     0
 CUBETEXP float32      s     0
MEANTRANS float32            0
   ISBEST    bool            0
CATSOURCE   int16            0
   Z_SPEC float32          286
Table info: 

<Table length=204110>
     name      dtype    unit     class    
------------- ------- ------- ------------
       CATAID   int32         MaskedColumn
           RA float64     deg       Column
          Dec float64     deg       Column
            Z float32               Column
       Rpetro float32     mag       Column
 DM_100_25_75 float32     mag       Column
  SURVEY_CODE   int16         MaskedColumn
       SigErr float32  km / s       Column
      GroupID   int32         MaskedColumn
  RankIterCen   int16         MaskedColumn
   SepIterC

In [2]:
# Remove all ISBEST == FALSE entries
isNOTbest = list()
for i in range(len(sami_cube_list)):
    if sami_cube_list['ISBEST'][i] == False:
        isNOTbest.append(i)
print(len(isNOTbest),"bad entries detected.")
sami_cube_list.remove_rows(isNOTbest)
print("Removal complete, original table now has",len(sami_cube_list),"rows.")

# Output GAMA galaxies that belong to groups
sami_cube_list.write("cube_list_ISBEST.fits",format='fits',overwrite=True)
print("Table written to file.")

# Check that code worked, each CATID is unique
unique_by_ID = unique(sami_cube_list,keys='CATID')
print(len(unique_by_ID))

467 bad entries detected.
Removal complete, original table now has 3245 rows.
Table written to file.
3245


In [18]:
# Go through the SAMI and GAMA CATID columns and store all matches in a list
sami_cube_list_ISBEST = Table.read('cube_list_ISBEST.fits',format='fits',memmap=True)

# Initialise lists to store ID's and row numbers
gama_ALL_galaxies_in_sami = list()
gama_ALL_galaxies_in_sami_row = list()
for i in range(len(gama_galaxies)):
    for j in range(len(sami_cube_list_ISBEST)):
        if gama_galaxies['CATAID'][i] == sami_cube_list_ISBEST['CATID'][j]:
            gama_ALL_galaxies_in_sami.append(gama_galaxies['CATAID'][i])
            gama_ALL_galaxies_in_sami_row.append(i)
print(len(gama_ALL_galaxies_in_sami),"matches found!")

# Store in a table
gama_CATAID_in_sami = Table([gama_ALL_galaxies_in_sami_row,gama_ALL_galaxies_in_sami], names =('Row_in_G3CGalv10','CATAID'))

# Write table to file
gama_CATAID_in_sami.write('GAMA_CATAID_in_SAMIv2.fits',format='fits')
print("Table written to file.")

2147 matches found!
Table written to file.
